In [264]:
dataset

DailyRate  MonthlyIncome  MonthlyRate  TotalWorkingYears  \
0          1102           5993        19479                  8   
1           279           5130        24907                 10   
2          1373           2090         2396                  7   
3          1392           2909        23159                  8   
4           591           3468        16632                  6   
...         ...            ...          ...                ...   
1465        884           2571        12290                 17   
1466        613           9991        21457                  9   
1467        155           6142         5174                  6   
1468       1023           5390        13243                 17   
1469        628           4404        10228                  6   

      YearsAtCompany  Attrition_Yes  
0                  6              1  
1                 10              0  
2                  0              1  
3                  8              0  
4                  2              0  
...              ...            ...  
1465               5              0  
1466               7              0  
1467               6              0  
1468               9              0  
1469               4              0  

[1470 rows x 6 columns]

In [265]:
dataset1=dataset.copy()

In [266]:
indep=dataset.drop('Attrition_Yes', axis=1)
dep=dataset['Attrition_Yes']

In [377]:
indep

DailyRate  MonthlyIncome  MonthlyRate  TotalWorkingYears  YearsAtCompany
0          1102           5993        19479                  8               6
1           279           5130        24907                 10              10
2          1373           2090         2396                  7               0
3          1392           2909        23159                  8               8
4           591           3468        16632                  6               2
...         ...            ...          ...                ...             ...
1465        884           2571        12290                 17               5
1466        613           9991        21457                  9               7
1467        155           6142         5174                  6               6
1468       1023           5390        13243                 17               9
1469        628           4404        10228                  6               4

[1470 rows x 5 columns]

In [378]:
X_train, X_test, y_train, y_test = train_test_split(indep, dep, stratify=dep,test_size=0.2, random_state=42)

In [394]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [395]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

In [396]:
y_train_res.value_counts()

Attrition_Yes
0    986
1    986
Name: count, dtype: int64

## Logistic Regression

In [412]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2'],
             'class_weight': ['balanced']} 

grid_lr = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_macro') 
   
# fitting the model for grid search 
grid_lr.fit(X_train_scaled, y_train_res)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'], 'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_macro', verbose=3)

In [413]:
y_pred = grid_lr.predict(X_test_scaled)

In [414]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
 
from sklearn.metrics import classification_report
report=classification_report(y_test, y_pred)

In [415]:
cm

array([[133, 114],
       [ 15,  32]], dtype=int64)

In [416]:
print(report)

              precision    recall  f1-score   support

           0       0.90      0.54      0.67       247
           1       0.22      0.68      0.33        47

    accuracy                           0.56       294
   macro avg       0.56      0.61      0.50       294
weighted avg       0.79      0.56      0.62       294



## Decision Tree

In [409]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': [None,'sqrt','log2'],
              'splitter':['best','random'],
              'class_weight': ['balanced']} 

grid_dt = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_macro') 
   
# fitting the model for grid search 
grid_dt.fit(X_train_scaled, y_train_res)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_macro', verbose=3)

In [410]:
y_pred = grid_dt.predict(X_test_scaled) 

from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test, y_pred)

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)

In [411]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.84      0.72      0.78       247
           1       0.15      0.26      0.19        47

    accuracy                           0.65       294
   macro avg       0.49      0.49      0.48       294
weighted avg       0.73      0.65      0.68       294



## Random Forest

In [405]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': [None,'sqrt','log2'],
              'n_estimators':[100,200],
              'class_weight': ['balanced']}

grid_RF = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_macro') 
   
# fitting the model for grid search 
grid_RF.fit(X_train_scaled, y_train_res)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [100, 200]},
             scoring='f1_macro', verbose=3)

In [406]:
y_pred = grid_RF.predict(X_test_scaled) 

from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_test, y_pred)

# print classification report 
from sklearn.metrics import classification_report
clf_report1 = classification_report(y_test, y_pred)

In [407]:
print(cm2)

[[191  56]
 [ 31  16]]


In [408]:
print(clf_report1)

              precision    recall  f1-score   support

           0       0.86      0.77      0.81       247
           1       0.22      0.34      0.27        47

    accuracy                           0.70       294
   macro avg       0.54      0.56      0.54       294
weighted avg       0.76      0.70      0.73       294



## SVM

In [351]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'kernel': ['poly', 'linear','sigmoid'],
    'gamma': ['scale', 'auto'],
    'C': [100, 3000],
    'class_weight': [None, 'balanced']
}
grid_svm = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid_svm.fit(X_train_scaled, y_train_res)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [100, 3000], 'class_weight': [None, 'balanced'],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'linear', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [352]:
y_pred = grid_svm.predict(X_test_scaled) 

from sklearn.metrics import confusion_matrix
cm3 = confusion_matrix(y_test, y_pred)

# print classification report 
from sklearn.metrics import classification_report
clf_report2 = classification_report(y_test, y_pred)

In [353]:
print(clf_report2)

              precision    recall  f1-score   support

           0       0.92      0.54      0.68       255
           1       0.19      0.69      0.29        39

    accuracy                           0.56       294
   macro avg       0.55      0.61      0.49       294
weighted avg       0.82      0.56      0.63       294



In [354]:
print(cm3)

[[137 118]
 [ 12  27]]


## KNN

In [345]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'euclidean', 'manhattan'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]
}

grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, refit = True, scoring='f1_weighted', verbose=3, n_jobs=-1)

grid_knn.fit(X_train_scaled, y_train_res)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [346]:
y_pred = grid_knn.predict(X_test_scaled) 

from sklearn.metrics import confusion_matrix
cm4 = confusion_matrix(y_test, y_pred)

# print classification report 
from sklearn.metrics import classification_report
clf_report3 = classification_report(y_test, y_pred)

In [347]:
print(clf_report3)

              precision    recall  f1-score   support

           0       0.90      0.66      0.76       255
           1       0.19      0.54      0.29        39

    accuracy                           0.64       294
   macro avg       0.55      0.60      0.52       294
weighted avg       0.81      0.64      0.70       294



In [348]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.3],
    'max_depth': [3, 5],
    'gamma': [0, 1, 5],                   
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'grow_policy': ['depthwise', 'lossguide']
}

grid_xgb = GridSearchCV(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),param_grid=param_grid_xgb,scoring='f1',refit=True,verbose=3,n_jobs=-1,cv=5)

grid_xgb.fit(X_train_res, y_train_res)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraint...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1], 'gamma': [0, 1, 5],
                         'grow_policy': ['depthwise', 'lossguide'],
                         'learning_rate': [0.1, 0.3], 'max_depth': [3, 5],
                         'min_child_weight': [1, 3], 'n_estimators': [100, 200],
                         'subsample': [0.8, 1]},
             scoring='f1', verbose=3)

In [349]:
y_pred = grid_xgb.predict(X_test_scaled) 

from sklearn.metrics import confusion_matrix
cm5 = confusion_matrix(y_test, y_pred)

# print classification report 
from sklearn.metrics import classification_report
clf_report4 = classification_report(y_test, y_pred)

In [350]:
print(clf_report4)

              precision    recall  f1-score   support

           0       0.95      0.16      0.28       255
           1       0.15      0.95      0.26        39

    accuracy                           0.27       294
   macro avg       0.55      0.56      0.27       294
weighted avg       0.85      0.27      0.28       294



In [211]:
import pickle
filename = 'final_RF_Grid.sav'
pickle.dump(grid_RF,open(filename,'wb'))